In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px

from morphomics.io.io import save_obj, load_obj
from utils import load_toml, run_toml, get_2d, mask_pi, get_base

from plot import plot_2d, plot_pi, plot_dist_matrix

In [ ]:
path = 'results/vae/trained_vae'
my_pip = load_obj(path)
vae_pip = my_pip.metadata
mf = my_pip.morphoframe['v1_pi']
# Reset index and store the old index in a new column
mf = mf.reset_index()  # Resets the index and adds the old index as a column
# Rename the old index column to 'old_idcs'
mf.rename(columns={'index': 'old_idcs'}, inplace=True)
pis = mf['pi']
pi_example = pis.iloc[0]

In [ ]:
def get_base(pi, pixes_tokeep):
    pi_full = np.zeros_like(pi_example)
    pi_full[pixes_tokeep] = pi
    return pi_full

# Plot one example

In [ ]:
plot_pi(pi_example)

# Check link to number of branches

In [ ]:
mf["pi_area"] = mf['pi'].apply(lambda pi: pi.sum())
fig = px.scatter(mf, x='nb_bars', y='pi_area')
fig.show()

# Distance Matrix of PI

In [ ]:
# Create a new column for the condition (Model + Sex)
mf['Condition'] = mf['Model'] + "-" + mf['Sex']
# Sort by condition
mf_sorted = mf.sort_values(by='Condition').reset_index(drop=True)

In [ ]:
# Extract vectors
pi_vectors = np.vstack(mf_sorted['pi'].values)  # Convert list of arrays to 2D array
# Compute pairwise Euclidean distance matrix
distance_matrix = squareform(pdist(pi_vectors, metric='euclidean'))
plot_dist_matrix(mf_sorted, distance_matrix)


In [ ]:
# Step 1: Compute the mean over each row
row_means = distance_matrix.mean(axis=1)
plt.plot(row_means)

# Analyse of the far PI

In [ ]:
# Step 2: Get the indices of the top 5 maximum values
far_top_5_idx = np.argsort(row_means)[-5:][::-1]  # Sort and take the top 5 indices (in descending order)

print("Indices of the top 5 maximum values:", far_top_5_idx)
print(row_means[far_top_5_idx])

In [ ]:
far_mf = mf_sorted.iloc[far_top_5_idx]

In [ ]:
from morphomics.view.view import neuron
for i, row in far_mf.iterrows():
    neuron(row['cells'])
    plot_pi(row['pi'])
    print(row['nb_bars'])

In [ ]:
far_mf

In [ ]:
small_top_5_idx = np.argsort(list(mf_sorted['nb_bars']))[:5] 
little_top_5_idx = np.argsort(list(mf_sorted['max_length_bar']))[:5]  # Sort and take the top 5 indices (in descending order)
 # Sort and take the top 5 indices (in descending order)
print('mg with less bars')
print(mf_sorted['nb_bars'].iloc[small_top_5_idx])
print('')
print('mg with smallest bars')
print(mf_sorted['max_length_bar'].iloc[little_top_5_idx])
print('')
print('mg with far pi')
print(far_mf[['nb_bars','max_length_bar']])


In [ ]:
mf_sorted_f = mf_sorted[mf_sorted['Sex']=='F'].reset_index(drop=True)
# Extract vectors
pi_vectors_f = np.vstack(mf_sorted_f['pi'].values)  # Convert list of arrays to 2D array
# Compute pairwise Euclidean distance matrix
distance_matrix_f = squareform(pdist(pi_vectors_f, metric='euclidean'))
plot_dist_matrix(mf_sorted_f, distance_matrix_f)

In [ ]:
row_means_f = distance_matrix_f.mean(axis=1)
plt.plot(row_means_f)
print(np.median(row_means_f))

In [ ]:
mf_sorted_m = mf_sorted[mf_sorted['Sex']=='M'].reset_index(drop=True)
# Extract vectors
pi_vectors_m = np.vstack(mf_sorted_m['pi'].values)  # Convert list of arrays to 2D array
# Compute pairwise Euclidean distance matrix
distance_matrix_m = squareform(pdist(pi_vectors_m, metric='euclidean'))
plot_dist_matrix(mf_sorted_m, distance_matrix_m)


In [ ]:
row_means_m = distance_matrix_m.mean(axis=1)
plt.plot(row_means_m)
print(np.median(row_means_m))

# Apply Threshold

In [ ]:
pixes_tokeep = vae_pip['pixes_tokeep']
pis_threshold = pis.apply(lambda pi: mask_pi(pi, pixes_tokeep)[0])
pis_filtered = pis.apply(lambda pi: mask_pi(pi, pixes_tokeep)[1])

In [ ]:
pi_th_example = pis_threshold.iloc[0]

plot_pi(pi_th_example, is_log = False)

# Apply Scaler

In [ ]:
standardizer = vae_pip['standardizer']
pis_filtered_arr = np.vstack(pis_filtered)
pis_scaled = standardizer.transform(pis_filtered_arr)

In [ ]:
pi_scaled_full_example = get_base(pis_scaled[0], pixes_tokeep)
plot_pi(pi_scaled_full_example, is_log = False)

# Apply PCA

In [ ]:
pca = vae_pip['fitted_pca_vae'][0]

In [ ]:
pis_pca = pca.transform(pis_scaled)

plot the pca 

In [ ]:
mf_pca = mf[['Layer', 'Model', 'Sex']]

In [ ]:
pis_pca_2d = pis_pca[:,[0,1]]
mf_pca['pi_pca_2d'] = list(pis_pca_2d)
plot_2d(mf_pca, 'pi_pca_2d', title = 'pca pi', conditions = ['Model', 'Sex'], name = None)

In [ ]:
pis_pca_2d = pis_pca[:,[1,2]]
mf_pca['pi_pca_2d'] = list(pis_pca_2d)
plot_2d(mf_pca, 'pi_pca_2d', title = 'pca pi', conditions = ['Model', 'Sex'], name = None)

explained variance per pc

In [ ]:
# Assuming `pca` is your fitted PCA model (e.g., from sklearn.decomposition.PCA)
explained_variance_ratio = pca.explained_variance_ratio_

# Cumulative variance explained by the first 20 PCs
cumulative_variance_64 = np.sum(explained_variance_ratio[:64])

print(f"Variance explained by the first 64 PCs: {cumulative_variance_64:.4f}")


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(explained_variance_ratio)[:64], marker='o', linestyle='--', label='Cumulative Variance')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by PCA Components')
plt.grid()
plt.legend()
plt.show()

pc selection

In [ ]:
mf_pca['pi_pca'] = list(pis_pca)
mf_pca['Condition'] = mf['Condition']
mf_pca_kxa = mf_pca[mf_pca['Model'].isin(['1xKXA_4h', '1xSaline_4h'])]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE

# Extract features and target variable
X = np.array(mf_pca_kxa['pi_pca'].to_list())  # 64 features in each row
y = mf_pca_kxa['Condition']

# Encode the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Initialize models
rf = RandomForestClassifier(random_state=42)
svm = SVC(kernel='linear', random_state=42)

# Cross-validation setup: 20 trials with 75% training / 25% testing
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.25, random_state=42)

In [ ]:
# Function to calculate feature importance for RandomForest
def get_rf_importance(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model.feature_importances_


# Function to calculate RFE and get top features
def get_rfe_top_features(model, X, y, n_features_to_select=10):
    rfe = RFE(estimator=model, n_features_to_select=n_features_to_select)
    rfe.fit(X, y)
    return rfe.support_, rfe.ranking_

In [ ]:
# Initialize lists to store accuracy and feature importances
rf_accuracies = []
rf_importances = []
# Perform the 20 trials
for train_idx, test_idx in sss.split(X, y_encoded):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
    
    # Train and evaluate RandomForest
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_accuracy = np.mean(rf_pred == y_test)
    rf_accuracies.append(rf_accuracy)
    rf_importances.append(rf.feature_importances_)

# Get average accuracy across all trials
rf_avg_accuracy = np.mean(rf_accuracies)

# Get average feature importance across all trials
rf_avg_importance = np.mean(rf_importances, axis=0)
# Get sorted indices based on importance (descending)
rf_sorted_idx = np.argsort(rf_avg_importance)[::-1]

In [ ]:
# Get RFE top features
rf_rfe_support, rf_rfe_ranking = get_rfe_top_features(rf, X, y_encoded)


In [ ]:

# Print feature importances and accuracies
print("Top 10 Features Based on Average Importance:")

print("\nRandom Forest Feature Importance (Top 10):")
for idx in rf_sorted_idx[:10]:
    print(f"Feature {idx} with importance: {rf_avg_importance[idx]}")
print(f"Random Forest Accuracy: {rf_avg_accuracy * 100:.2f}%\n")

# Print RFE selected features
print("\nTop 10 Features Based on RFE:")

print("Random Forest RFE Selected Features:")
rf_rfe_selected = np.where(rf_rfe_support)[0]
print(f"Selected Features: {rf_rfe_selected}")


# Optional: Display bar charts for feature importance
plt.figure(figsize=(12, 6))

# Random Forest
plt.bar(range(len(rf_avg_importance)), rf_avg_importance[rf_sorted_idx])
plt.xticks(range(len(rf_avg_importance)), rf_sorted_idx, rotation=90)
plt.title('Random Forest Feature Importance')

plt.tight_layout()
plt.show()


plot weight pcs (load)

In [ ]:
###  rf_rfe_selected or rf_sorted_idx or svm_rfe_selected

# Get the PCA components (eigenvectors)
loadings = pca.components_  # Shape: (n_components, n_features)

# If feature names are available
feature_names = [f'Feature{i+1}' for i in range(loadings.shape[1])]  # Replace with actual feature names if available

# Convert to a DataFrame for better readability
pc_load_df = pd.DataFrame(loadings, columns=feature_names, index=[f'PC{i+1}' for i in range(len(loadings))])

In [ ]:
for i in rf_sorted_idx[:5]:
    pc_load_i_full = get_base(pc_load_df.iloc[i], pixes_tokeep)
    plot_pi(pc_load_i_full, is_log = False)

# UMAP with best features (PCs)


only kxa

In [ ]:
#  rf_rfe_selected or rf_sorted_idx or svm_rfe_selected
selected_pcs = rf_rfe_selected
k = 8

In [ ]:
mf_pca_kxa['pipcaselected'] = mf_pca_kxa['pi_pca'].apply(lambda vec: vec[selected_pcs[:k]])
parameters_filepath = "h_pca_umap.toml"
parameters = load_toml(parameters_filepath=parameters_filepath)
my_pip_kxa = run_toml(parameters=parameters, morphoframe = {'v1_pi_pca_umap': mf_pca_kxa}) 

In [ ]:
plot_2d(df = my_pip_kxa.morphoframe['v1_pi_pca_umap'],
         feature = 'umap', 
         title = 'pi umap best pcs', 
         conditions = ['Model', 'Sex'], 
         name = None)

full

In [ ]:
mf_pca['pipcaselected'] = mf_pca['pi_pca'].apply(lambda vec: vec[selected_pcs[:k]])
parameters_filepath = "h_pca_umap.toml"
parameters = load_toml(parameters_filepath=parameters_filepath)
my_pip_full = run_toml(parameters=parameters, morphoframe = {'v1_pi_pca_umap': mf_pca}) 

In [ ]:
plot_2d(df = my_pip_full.morphoframe['v1_pi_pca_umap'],
         feature = 'umap', 
         title = 'pi umap best pcs', 
         conditions = ['Model', 'Sex'], 
         name = None)